Readme

This project aims to spot opportunities to buy from hedonism wines when a price item is the lowest.
To achieve this I will use:

- A duckdb instance and then RDS to save the data every day
- streamlit to visualise the results of a few simple queries
- heroku to host a simple app
- A scheduler to download a daily copy of the hedonism wines full stock list
- create an alert (tech tbc) to send an email when an items drops by a certain percentage

### To Do

1. Save files in S3
2. Load files into an RDS table
3. Connect RDS to Heroku as external db

In [1]:
import pandas as pd
import numpy as np
import requests
import streamlit as st
import os
from io import StringIO
from datetime import datetime
from pandasql import sqldf
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import duckdb
import altair as alt

### Import multiple static files

In [3]:
import pandas as pd
import os

# Directory containing the CSV files
folder_path = '/Users/MacUser/hedonism-wines_app/data/'

# List to store DataFrames
dfs = []

# Loop through files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        # Extract date from file name
        file_date = file_name.split('.')[0]  # remove '.csv' extension
        file_date = file_date.split('_')      # split by underscore

        import_date = '-'.join(file_date)    # join with hyphen

        # Construct the full file path
        file_path = os.path.join(folder_path, file_name)
        
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        
        
        # Add import_date column with the extracted date
        df['import_date'] = import_date
        
        # Append the DataFrame to the list
        dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(dfs, ignore_index=True)

# Now you have a single DataFrame 'combined_df' containing data from all CSV files.


In [26]:
combined_df

Code                                              Title  Size  \
0       HED0017       Baron Rothschild Lafite Tres Vieille Reserve  70cl   
1       HED0037                                     Louis XIII      70cl   
2       HED0059                                 Ardbeg 17 Year Old  70cl   
3       HED0062                           Ardbeg Lord of the Isles  70cl   
4       HED0068                               Balvenie 40 Year Old  70cl   
...         ...                                                ...   ...   
31397  HED21759  Port Ellen 41 Year Old Prima & Ultima Third Re...  70cl   
31398  HED21760  Lagavulin 25 Year Old Prima & Ultimate Fourth ...  70cl   
31399  HED21761  Port Ellen 43 Year Old Prima & Ultimate Fourth...  70cl   
31400  HED47802                     Rossville Union Rye 7 Year Old  70cl   
31401  HED00101  Ardbeg 15 Year Old Single Oloroso Cask 1322 Ex...  70cl   

         Style        Country    Group  Available  Price (GBP)  \
0      Spirits         France   Cognac          2       1350.0   
1      Spirits         France   Cognac          4       2950.0   
2      Spirits       Scotland   Whisky          2        595.0   
3      Spirits       Scotland   Whisky          2       2400.0   
4      Spirits       Scotland   Whisky          4       7250.0   
...        ...            ...      ...        ...          ...   
31397  Spirits       Scotland   Whisky          1       9550.0   
31398  Spirits       Scotland   Whisky          1       2400.0   
31399  Spirits       Scotland   Whisky          1      15000.0   
31400  Spirits  United States  Whiskey          3         90.0   
31401  Spirits       Scotland   Whisky          1       1100.0   

                      import_date  
0      full-stock-list-2024-03-23  
1      full-stock-list-2024-03-23  
2      full-stock-list-2024-03-23  
3      full-stock-list-2024-03-23  
4      full-stock-list-2024-03-23  
...                           ...  
31397  full-stock-list-2024-03-24  
31398  full-stock-list-2024-03-24  
31399  full-stock-list-2024-03-24  
31400  full-stock-list-2024-03-24  
31401  full-stock-list-2024-03-24  

[31402 rows x 9 columns]

In [27]:
combined_df.rename(columns={'Code':'code',
                  'Title':'title',
                  'Size':'size',
                  'Style':'style',
                  'Country':'country',
                  'Group':'type',
                  'Available':'availability',
                  'Price (GBP)': 'price_gbp'},inplace=True)

In [28]:
combined_df['import_date'] = combined_df['import_date'].str[-10:]

# Convert the 'import_date' column to datetime objects
combined_df['import_date'] = pd.to_datetime(combined_df['import_date'])

In [29]:
combined_df['title'] = combined_df['title'].str.strip()

In [30]:
combined_df['url'] = 'https://hedonism.co.uk/product/' + combined_df['title'].str.replace(' ', '-').str.lower() + '-whisky'

In [9]:
pd.set_option('display.max_columns', None)  # Display all columns

In [13]:
#new_df = combined_df.head(10)

In [14]:
#new_df.to_csv('output.csv', index=False)

In [31]:
combined_df['title'][combined_df['code']=='HED0037']

1        Louis XIII
10494    Louis XIII
20963    Louis XIII
Name: title, dtype: object

### Importing a static file

In [2]:
# df = pd.read_csv('/Users/MacUser/Downloads/full-stock-list_2024_03_23.csv')

In [3]:
# df['import_date'] = '2024-03-23'

In [27]:
# df.rename(columns={'Code':'code',
#                   'Title':'title',
#                   'Size':'size',
#                   'Style':'style',
#                   'Country':'country',
#                   'Group':'type',
#                   'Available':'availability',
#                   'Price (GBP)': 'price_gbp'},inplace=True)

### Importing files by calling the API

In [5]:
url = 'https://hedonism.co.uk/sites/default/files/full-stock-list.csv'

# Define headers
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Accept-Language': 'en-US,en;q=0.9',
}

# Make request with headers
response = requests.get(url, headers=headers)

# Check if request was successful
if response.status_code == 200:
    # Read CSV from response content
    csv_data = StringIO(response.text)
    df = pd.read_csv(csv_data)
else:
    print("Failed to fetch data:", response.status_code)
    
df.head()  # Display first few rows of the DataFrame

Code                                         Title  Size    Style  \
0  HED0017  Baron Rothschild Lafite Tres Vieille Reserve  70cl  Spirits   
1  HED0037                                Louis XIII      70cl  Spirits   
2  HED0059                            Ardbeg 17 Year Old  70cl  Spirits   
3  HED0062                      Ardbeg Lord of the Isles  70cl  Spirits   
4  HED0068                          Balvenie 40 Year Old  70cl  Spirits   

    Country   Group  Available  Price (GBP)  
0    France  Cognac          1       1350.0  
1    France  Cognac          4       2950.0  
2  Scotland  Whisky          2        595.0  
3  Scotland  Whisky          2       2400.0  
4  Scotland  Whisky          4       7250.0

In [6]:
# Get today's date
today_date = datetime.today().date()

# Add today's date to a new column in the DataFrame
df['import_date'] = today_date

In [7]:
df.rename(columns={'Code':'code',
                  'Title':'title',
                  'Size':'size',
                  'Style':'style',
                  'Country':'country',
                  'Group':'type',
                  'Available':'availability',
                  'Price (GBP)': 'price_gbp'},inplace=True)

In [8]:
df.head()

code                                         title  size    style  \
0  HED0017  Baron Rothschild Lafite Tres Vieille Reserve  70cl  Spirits   
1  HED0037                                Louis XIII      70cl  Spirits   
2  HED0059                            Ardbeg 17 Year Old  70cl  Spirits   
3  HED0062                      Ardbeg Lord of the Isles  70cl  Spirits   
4  HED0068                          Balvenie 40 Year Old  70cl  Spirits   

    country    type  availability  price_gbp import_date  
0    France  Cognac             1     1350.0  2024-03-25  
1    France  Cognac             4     2950.0  2024-03-25  
2  Scotland  Whisky             2      595.0  2024-03-25  
3  Scotland  Whisky             2     2400.0  2024-03-25  
4  Scotland  Whisky             4     7250.0  2024-03-25

### Queries using sqldf engine

In [60]:
# Create an in-memory SQLite database engine
#engine = create_engine('sqlite://', echo=False)

In [61]:
# Convert the DataFrame to a SQL table in the in-memory SQLite database
#df.to_sql('mydata', con=engine, if_exists='replace', index=False)

In [62]:
# Execute the SQL query and fetch all results
#results = engine.execute("""
#                        SELECT COUNT (*), SUM (price_gbp), AVG(price_gbp)
#                        FROM mydata 
#                        """).fetchall()
#                        
# Convert the results to a DataFrame
#result_df = pd.DataFrame(results)

### Queries using sqlalchemy and duck db to create an initial table

In [24]:
# Specify the file path for the DuckDB database
db_path = '/Users/MacUser/hedonism-wines_app/database.db'  # Example path, replace with your desired path

In [25]:
# # Establish a connection to an in-memory DuckDB database
conn = duckdb.connect(database=db_path, read_only=False)

In [32]:
# # Create a DuckDB table from the DataFrame
#conn.register('stocks_table', df)
combined_df.to_sql(name='stocks_table', con=conn, index=False, if_exists='replace')

/Users/MacUser/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


-1

In [33]:
# # Commit the transaction to save changes
conn.commit()

In [28]:
# Execute SQL queries to create a table only for whisky records
results = conn.execute("""SELECT COUNT (*), import_date FROM stocks_table GROUP BY import_date
                """).fetchdf()

# Convert the results to a DataFrame
result_df = pd.DataFrame(results)

In [29]:
result_df

count_star() import_date
0         10493  2024-03-23
1         10440  2024-03-24
2         10421  2024-03-25

In [34]:
# # Close connection to the db
conn.close()

#### Check existing tables

In [48]:
# # # Execute the SHOW TABLES command
# result = conn.execute("SHOW TABLES")

# # # Fetch all table names
# table_names = result.fetchall()

# # # Print the list of tables
# print("List of tables:")
# for table_name in table_names:
#      print(table_name[0])

# # # Close the connection
# conn.close()

List of tables:
stocks_table


#### Drop unnecessary Views

In [9]:
# # Drop a table (replace 'table_name' with the name of the table you want to drop)
# conn.execute("DROP VIEW IF EXISTS stocks")
conn.execute("DROP TABLE IF EXISTS stocks_table")
conn.execute("DROP TABLE IF EXISTS whisky_stocks_table")

### Inserting new data into duckdb

In [35]:
# Specify the file path for the DuckDB database
db_path = '/Users/MacUser/hedonism-wines_app/database.db'  # Example path, replace with your desired path

In [36]:
# Establish a connection to an in-memory DuckDB database
conn = duckdb.connect(database=db_path, read_only=False)

In [27]:
today_date = datetime.now().strftime('%Y-%m-%d')

In [32]:
# Execute DELETE query
delete_query = f"DELETE FROM stocks_table WHERE import_date = '{today_date}'"

# Execute the query
conn.execute(delete_query)

# Commit the changes
conn.commit()

# Close connection
conn.close()

In [ ]:
conn = duckdb.connect(database=db_path, read_only=False)

In [12]:
# Get the column names from the DataFrame
columns = list(df.columns)

In [13]:
# Generate the list of column names for the INSERT INTO statement
column_names = ", ".join(columns)

In [14]:
# Generate the list of parameter placeholders (?, ?, ?) for the VALUES clause
parameter_placeholders = ", ".join(["?" for _ in range(len(columns))])

In [15]:
# Convert the DataFrame to records list
records = df.values.tolist()

In [16]:
# Define the name of your existing table
table_name = 'stocks_table'

In [17]:
# Construct the SQL INSERT INTO statement dynamically
sql_insert = f"INSERT INTO {table_name} ({column_names}) VALUES ({parameter_placeholders})"

In [18]:
# Execute the INSERT statement
conn.executemany(sql_insert, records)

# Commit the transaction (optional, depending on your needs)
conn.commit()

In [37]:
# Execute SQL queries to create a table only for whisky records
conn.execute("""CREATE OR REPLACE TABLE whisky_stocks_table AS 
                SELECT * FROM stocks_table 
                WHERE type = 'Whisky'""")

In [31]:
# Close the connection
conn.close()

### Expection of duplicated entries with different codes but same title

In [50]:
# Execute SQL queries to create a table only for whisky records
results = conn.execute("""SELECT DISTINCT code, title, price_gbp, url
                          FROM whisky_stocks_table
                          WHERE price_gbp = 4100.0 
                          OR price_gbp = 4950.0
                """).fetchdf()

# Convert the results to a DataFrame
result_df = pd.DataFrame(results)

In [51]:
result_df

code                                  title  price_gbp  \
0   HED3548  Macallan 25 Year Old Anniversary Malt     4950.0   
1  HED87923  Macallan 25 Year Old Anniversary Malt     4100.0   

                                                 url  
0  https://hedonism.co.uk/product/macallan-25-yea...  
1  https://hedonism.co.uk/product/macallan-25-yea...

### Visualisation

In [2]:
# Specify the file path for the DuckDB database
db_path = '/Users/MacUser/hedonism-wines_app/database.db'  # Example path, replace with your desired path

# Establish a connection to an in-memory DuckDB database
conn = duckdb.connect(database=db_path, read_only=False)

#### Show discounted items

In [3]:
# conn.execute("""
# UPDATE database.whisky_stocks_table 
# SET price_gbp = 24500 
# WHERE code = 'HED10582' 
# AND import_date = CURRENT_DATE()""")

In [4]:
# # # Commit the transaction to save changes
# conn.commit()

In [5]:
# Execute SQL queries to create a table only for whisky records
results = conn.execute("""

WITH current_price AS (
    SELECT code, price_gbp, import_date, title, url
    FROM whisky_stocks_table
    WHERE import_date = CURRENT_DATE()
),
minimum_price AS (
    SELECT code, price_gbp, import_date
    FROM (
        SELECT code, 
               RANK() OVER (PARTITION BY code ORDER BY price_gbp ASC) rank,
               price_gbp,
               import_date
        FROM whisky_stocks_table
    ) ranked
    WHERE rank = 1
),
previous_price AS (
    SELECT code, price_gbp, min(import_date) import_date
    FROM (
        SELECT code, 
               RANK() OVER (PARTITION BY code ORDER BY price_gbp ASC) rank,
               price_gbp,
               import_date import_date
        FROM whisky_stocks_table
    ) ranked
    WHERE rank = 2
    GROUP BY code, price_gbp

),
output AS (
    SELECT  c.code, 
            c.title,
            c.url,
            c.price_gbp as current_minimum_price, 
            c.import_date as current_date,
            p.price_gbp as previous_price,
            p.import_date as previous_date,
            p.price_gbp - m.price_gbp as price_diff,
            ((p.price_gbp - m.price_gbp)/p.price_gbp)*100 AS perc_saving
    FROM current_price c 
    JOIN minimum_price m ON c.code = m.code AND c.price_gbp = m.price_gbp
    JOIN previous_price p ON c.code = p.code
)
SELECT * FROM output WHERE price_diff > 0

                """).fetchdf()

# Convert the results to a DataFrame
result_df_1 = pd.DataFrame(results)
result_df_1

Empty DataFrame
Columns: [code, title, url, current_minimum_price, current_date, previous_price, previous_date, price_diff, perc_saving]
Index: []

In [ ]:
# Add a title to the Streamlit app
st.title('Discounts')

In [5]:
result_df_1

min_price  max_price  price_diff  discount_perc    max_date  \
0     4100.0     4950.0       850.0          17.17  2024-03-24   

                                       title  
0  Macallan 25 Year Old Anniversary Malt

In [6]:
# Create Altair chart with tooltips
chart = alt.Chart(result_df_1).mark_bar().encode(
    x='title',
    y='min_price',
    tooltip=['title', 'min_price']
).interactive()

# Display the chart using Streamlit Vega-Lite
st.altair_chart(chart
                #, use_container_width=True
               )

2024-03-24 23:00:14.189 
  command:

    streamlit run /Users/MacUser/opt/anaconda3/lib/python3.9/site-packages/ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()

#### Show stocks and median value

In [7]:
# Execute SQL queries to create a table only for whisky records
results = conn.execute("""SELECT COUNT (*) stock_count,
                          MEDIAN (price_gbp) median_price,
                          import_date
                          FROM whisky_stocks_table 
                          GROUP BY import_date
                          ORDER BY 3 DESC
                """).fetchdf()

# Convert the results to a DataFrame
result_df_2 = pd.DataFrame(results)

In [11]:
# Convert import_date to datetime
result_df_2['import_date'] = pd.to_datetime(result_df_2['import_date'])

# Extract date part
result_df_2['import_date'] = result_df_2['import_date'].dt.date

result_df_2['import_date'] = result_df_2['import_date'].astype(str).str[:10]

In [ ]:
# Add a title to the Streamlit app
st.title('Stock and Median Price Check')

In [13]:
result_df_2

stock_count  median_price import_date
0         1342         233.5  2024-03-23
1         1348         240.0  2024-03-24

In [68]:
# Create Altair chart
primary_y_axis = alt.Axis(title='Stock Count', grid=False)
secondary_y_axis = alt.Axis(title='Median Price', grid=False, orient='right')

# Line chart for stock count
line_chart_stock_count = alt.Chart(result_df_2).mark_point(color='blue').encode(
    x='import_date',
    y=alt.Y('stock_count:Q', axis=primary_y_axis),
)

# Line chart for median price
line_chart_median_price = alt.Chart(result_df_2).mark_point(color='red').encode(
    x='import_date',
    y=alt.Y('median_price:Q', axis=secondary_y_axis),
)

# Combine both charts
combined_chart = line_chart_stock_count + line_chart_median_price

# Display the chart using Streamlit Vega-Lite
st.altair_chart(combined_chart, use_container_width=True)

#### Price Search

In [69]:
# Execute SQL queries to create a table only for whisky records
results = conn.execute("""SELECT MAX (price_gbp) price_gbp,
                          title
                          FROM whisky_stocks_table 
                          GROUP BY title
                          ORDER BY 1 DESC
                """).fetchdf()

# Convert the results to a DataFrame
result_df_3 = pd.DataFrame(results)

In [70]:
#result_df_3

price_gbp                                              title
0      700000.0         Macallan Red Collection Six Bottle Set    
1      370000.0                Macallan 60 Year Old Red Collection
2      140000.0                           Bowmore Black DB5 1964  
3       95000.0                                   Yamazaki Junshin
4       80600.0  Macallan Lalique Golden Age of Travel Aeroplan...
...         ...                                                ...
1342        5.0                          Brenne Cuvee Speciale 3cl
1343        4.9                                    Rock Oyster 3cl
1344        4.6                                   Kirkwall Bay 3cl
1345        4.5  Singleton of Dufftown Malt Master`s Selection 3cl
1346        3.0                               Paul John Edited 3cl

[1347 rows x 2 columns]

In [ ]:
# Add a title to the Streamlit app
st.title('Price Search')

In [71]:
# Create text input boxes for the left and right values of the slider
left_value = st.text_input('Enter left value:', value='0')
right_value = st.text_input('Enter right value:', value='5000')

# Create a text input box for filtering the title
title_filter = st.text_input('Enter title:', value='Yamazaki')

# Convert the input values to integers
left_value = int(left_value)
right_value = int(right_value)

# Create a slider for selecting the price range
price_range = st.slider('Select price range (GBP)', min_value=0, max_value=700000, value=(left_value, right_value), step=1000)

# Filter the DataFrame based on the selected price range

# Filter the DataFrame based on the selected price range and title filter
filtered_df = result_df_3[
    (result_df_3['price_gbp'] >= price_range[0]) & 
    (result_df_3['price_gbp'] <= price_range[1]) &
    (result_df_3['title'].str.contains(title_filter, case=False))
]

#filtered_df = result_df_3[(result_df_3['price_gbp'] >= price_range[0]) & (result_df_3['price_gbp'] <= price_range[1])]

# Create Altair chart with tooltips
chart = alt.Chart(filtered_df).mark_bar().encode(
    x=alt.X('title', sort='-y'),
    y='price_gbp',
    tooltip=['title', 'price_gbp']
).interactive()

# Display the chart using Streamlit Vega-Lite
st.altair_chart(chart
                #, use_container_width=True
               )


DeltaGenerator()

In [61]:
conn.close()

### Convert notebook into .py file

In [21]:
! jupyter nbconvert --to script hedonism_wines.ipynb --output-dir /Users/MacUser/hedonism-wines_app

[NbConvertApp] Converting notebook hedonism_wines.ipynb to script
[NbConvertApp] Writing 12154 bytes to /Users/MacUser/hedonism-wines_app/hedonism_wines.py


### GenAI Part

In [23]:
# import openai
# import os
# from dotenv import load_dotenv

In [24]:
# # Load environment variables from .env file
# load_dotenv()

# # Access the API key from the environment
# api_key = os.getenv("OPENAI_API_KEY")

In [25]:
# import os
# from openai import OpenAI

# client = OpenAI(
#     # This is the default and can be omitted
#     api_key=os.environ.get("OPENAI_API_KEY"),
# )

# chat_completion = client.chat.completions.create(
#     messages=[
#         {
#             "role": "user",
#             "content": "Say this is a test",
#         }
#     ],
#     model="gpt-3.5-turbo",
# )

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

### db check

In [3]:
# Specify the file path for the DuckDB database
db_path = '/Users/MacUser/hedonism-wines_app/database.db'  # Example path, replace with your desired path

# Establish a connection to an in-memory DuckDB database
conn = duckdb.connect(database=db_path, read_only=False)

In [6]:
# Execute SQL queries to create a table only for whisky records
results = conn.execute("""SELECT COUNT (*),
                          import_date
                          FROM stocks_table 
                          GROUP BY import_date
                          ORDER BY 2 DESC
                """).fetchdf()

# Convert the results to a DataFrame
result_df = pd.DataFrame(results)

In [7]:
result_df

count_star() import_date
0         10419  2024-03-25
1         10436  2024-03-24
2         10493  2024-03-23